## Setup

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-13 05:25:25--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.30MB/s    in 0.7s    

2022-02-13 05:25:27 (1.30 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LoadingWirelessReview").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

## Extract

In [4]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz'
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"),header=True, sep=r'\t', inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [5]:
# Number of rows
df.count()

9002021

## Transform

In [6]:
# Get schema of dataframe
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
# create review_id_table
df_review_id_table = df.select(['review_id','customer_id','product_id','product_parent','review_date'])
df_review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543| 2015-08-31|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650| 2015-08-31|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154| 2015-08-31|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704| 2015-08-31|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [8]:
# Convert review_date column to date type
from pyspark.sql.functions import to_date
df_review_id_table = df_review_id_table.withColumn('review_date', to_date('review_date',"yyyy-MM-dd").alias('review_date'))
df_review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543| 2015-08-31|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650| 2015-08-31|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154| 2015-08-31|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704| 2015-08-31|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300| 2015-08-31|
|R11LOHEDYJALTN|   13334021|B00XVGJMDQ|     421688488| 2015-08-31|
|R3ALQVQB2P9LA7|   27520697|B00KQW1X1C|     554285554| 2015-08-31|
|R3MWLXLNO21PDQ|   48086021|B00IP1MQNK|     488006702| 2015-08-31|
|R2L15IS24CX0LI|   12738196|B00HVORET8|     389677711| 2015-08-31|
|R1DJ8976WPWVZU|   15867807|B00HX3G6J6|     299654876| 2015-08-31|
|R3MRWNNR8CBTB7|    1972249|B00U4NATNQ|     577878727| 2015-08-31|
|R1DS6DKTUXAQK3|   10956619|B00SZEFDH8|     654620704| 2015-08

In [9]:
df_review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [10]:
# Check for duplicates
if df_review_id_table.count() > df_review_id_table.dropDuplicates(['review_id','customer_id','product_id','product_parent','review_date']).count():
    raise ValueError('Data has duplicates')

In [11]:
# Create product table
df_products = df.select(['product_id', 'product_title'])
df_products.show(5, truncate=False)

+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                                                                                                                                                                                                                                                                                                                  |
+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
# Check Schema
df_products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [13]:
# Check for duplicates
if df_products.count() > df_products.dropDuplicates(['product_id']).count():
    raise ValueError('Data has duplicates')

ValueError: ignored

In [14]:
# Verify row count
df_products.count()

9002021

In [15]:
# drop duplicates of product id (is a unique primary key) and see new row count
df_products = df_products.dropDuplicates(['product_id'])
df_products.count()

906592

In [16]:
# verify format
df_products.show(5, truncate=False)

+----------+-------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                          |
+----------+-------------------------------------------------------------------------------------------------------+
|011040047X|Purple Hard Case Cover for Iphone 4 4s 4g with 3d Sculpture Design Blossom Rose Flower                 |
|0110400550|Pink & White 3d Melt Ice-cream Skin Hard Case Cover for Apple Iphone 4 4s Protect Cell                 |
|0133070565|Samsung Galaxy S III I9300 16gb Blue Unlocked GSM Phone no warranty                                    |
|0214614700|Cool Summer Breeze in the Ocean Beach Collection: iPhone4/4s Case with Embossment- Coral Sea           |
|0214714705|Cool Summer Breeze In The Ocean Beach Collection: iPhone4/4S Case with Embossment-The Star of the Ocean|
+----------+----------------------------------------------------

In [17]:
# create customer table and group by customer id to get count of time customer id appears in dataset (aka customer count)
df_customers = df.select(['customer_id'])
df_customers = df_customers.groupby('customer_id').count()

In [18]:
df_customers.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   46909180|    6|
|   42560427|    7|
|   43789873|    3|
|   22037526|    2|
|   34220092|    2|
+-----------+-----+
only showing top 5 rows



In [19]:
# rename column to correct name
df_customers = df_customers.withColumnRenamed('count', 'customer_count')
df_customers.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46909180|             6|
|   42560427|             7|
|   43789873|             3|
|   22037526|             2|
|   34220092|             2|
+-----------+--------------+
only showing top 5 rows



In [20]:
df_customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [21]:
# convert customer_count from long to integer
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

df_customers = df_customers.withColumn('customer_count', col('customer_count').cast('integer'))
df_customers.printSchema()
df_customers.show()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46909180|             6|
|   42560427|             7|
|   43789873|             3|
|   22037526|             2|
|   34220092|             2|
|   42801586|             1|
|    9565734|             2|
|   15829398|             1|
|   38247118|             1|
|   32478248|             2|
|   48114630|             1|
|   23085063|             1|
|   32787070|             3|
|   43515569|             1|
|    4919528|             2|
|    5088547|             2|
|   41852407|             3|
|   49703087|             1|
|   12713799|             1|
|   36728141|             8|
+-----------+--------------+
only showing top 20 rows



In [22]:
# Create vine_table
df_vine_table = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
df_vine_table.printSchema()
df_vine_table.show(5)

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|
|R15V54KBMTQWAY|          4|            0|          0|   N|
| RY8I449HNXSVF|          5|            0|          0|   N|
|R18TLJYCKJFLSR|          5|            0|          0|   N|
|R1NK26SWS53B8Q|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [23]:
# change type of star_rating from string to integer
df_vine_table = df_vine_table.withColumn('star_rating', col('star_rating').cast('integer'))
df_vine_table.printSchema()
df_vine_table.show()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|
|R15V54KBMTQWAY|          4|            0|          0|   N|
| RY8I449HNXSVF|          5|            0|          0|   N|
|R18TLJYCKJFLSR|          5|            0|          0|   N|
|R1NK26SWS53B8Q|          5|            0|          0|   N|
|R11LOHEDYJALTN|          5|            0|          0|   N|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|
|R2L15IS24CX0LI|          5|            0|          0|   N|
|R1DJ8976WPWVZU|          3|            0|          0|   N|
|R3

## Load

In [24]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://amazon-reviews.cdslqksvwrhx.ca-central-1.rds.amazonaws.com:5432/amazon-reviews"
config = {"user":"root",
          "password": "<password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table

df_review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
df_products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
# df_customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config) # first data set was uploaded so skipping this
df_vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)